# Chapter 10 The Factory

The factory is the last design pattern covered. The main objective of the factory is to provide a class that spits objects when called, and transforms strings into objects. With that it is able to provide an easy interface and also develop the open-closed principle.


## The main class


In [5]:
class PayOffFactory:
    def __init__(self):
        self.__TheCreatorFunctions = {}        
    
    def __del__(self):
        del self
    
    def RegisterPayOff(self, PayOffId, CreatorFunction):
        self.__TheCreatorFunctions[PayOffId] = CreatorFunction
        
    def CreatePayOff(self, PayOffId, Strike):
        if PayOffId not in self.__TheCreatorFunctions.keys():
            print(f'{PayOffId} is an unknown payoff')
            return None
        else:
            return self.__TheCreatorFunctions[PayOffId](Strike)   

The initial code adds a template class for handling the registrations, the code looks like this:

In [6]:
class PayOffHelper:    
    
    def __init__(self,PayOffId,PayOff):
        self.PayOff = PayOff        
        thePayOffFactory.RegisterPayOff(PayOffId,self.Create)
        

    def Create(self, Strike):
        return self.PayOff(Strike)
    

However this can be confusing and a little bit of a mess, this construction makes sense in strong typed languages such as C++ but in python this does not make sense, and it is way simpler to just call the method, since it is not needed the PayOffHelper (that in the C++ code was a template class). In C++ the template PayOffHelper allows to register payoffs without knowing the type (something that is not possible in the method of the Factory class), in python knowing the type is not needed, thus it makes no sense to really use it.


### Using thePayOffFactory

In [24]:
class PayOff():
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

class PayOffBridge():        
    def __del__(self):
        del self
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self = deepcopy(InnerPayOff)
        
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)

class PayOffPut(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(self.__Strike - Spot,0) 
    
class PayOffForward(PayOff,PayOffBridge):
    def __init__(self, Strike):
        self.__Strike = Strike
        
    def __call__(self, Spot):
        return Spot - Strike

In [7]:
global thePayOffFactory  # We just the global keyword to be accesible from anywhere
thePayOffFactory  = PayOffFactory()

Registering objects using the PayOffHelper

In [8]:
RegisterCall = PayOffHelper("call",PayOffCall)
RegisterPut = PayOffHelper("put",PayOffPut)

Now we use it for a very simple example.

In [13]:
Strike = 2
name = "call"
Spot = 3
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


Now we will do the same but registering it with the method of the factory.

In [18]:
del thePayOffFactory

In [20]:
global thePayOffFactory  # We just the global keyword to be accesible from anywhere
thePayOffFactory  = PayOffFactory()
thePayOffFactory.RegisterPayOff("call",PayOffCall)
thePayOffFactory.RegisterPayOff("put",PayOffPut)

In [21]:
Strike = 2
name = "call"
Spot = 3
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


So we can see that the result is the same.

### Exercises

**Exercise 10.1** Write a straddle class and register it with the factory.

The payoff of a straddle is the absolute difference between the strike and the spot.

Let's implement the class.

In [25]:
class PayOffStraddle(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return abs(self.__Strike - Spot) 

Now we will register it.

In [26]:
thePayOffFactory.RegisterPayOff("straddle",PayOffStraddle)

And then we try some easy examples.

In [27]:
Strike = 2
name = "straddle"
Spot = 3
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


In [28]:
Strike = 4
name = "straddle"
Spot = 3
thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
if thePayOFF != None:
    print(f'The payoff is: {thePayOFF(Spot)}')
    del thePayOFF

The payoff is: 1


**Exercise 10.2** Our class cannot handle a double digital as it needs two strikes. Work out a solution that will handle options with multiple parameters.

**Exercise 10.3** Integrate the factory with a Monte Carlo routine.